##Import Library Dan Menghubungkan Ke Google Drive

In [1059]:
import pandas as pd

In [1060]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


##Membaca Data

In [1061]:
path = '/content/drive/MyDrive/Colab Notebooks/unclean_smartwatch_health_data.csv'
data = pd.read_csv(path, low_memory=False)

In [1062]:
print(data)

      User ID  Heart Rate (BPM)  Blood Oxygen Level (%)    Step Count  \
0      4174.0         58.939776               98.809650   5450.390578   
1         NaN               NaN               98.532195    727.601610   
2      1860.0        247.803052               97.052954   2826.521994   
3      2294.0         40.000000               96.894213  13797.338044   
4      2130.0         61.950165               98.583797  15679.067648   
...       ...               ...                     ...           ...   
9995   1524.0         78.819386               98.931927   2948.491953   
9996   4879.0         48.632659               95.773035   4725.623070   
9997   2624.0         73.834442               97.945874   2571.492060   
9998   4907.0               NaN               98.401058   3364.788855   
9999   4113.0         70.063864               98.475606    544.696104   

     Sleep Duration (hours) Activity Level Stress Level  
0         7.167235622316564  Highly Active            1  
1      

In [1063]:
print(data.info())
print(data.isnull().sum())
print(data.head(25))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 7 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   User ID                 9799 non-null   float64
 1   Heart Rate (BPM)        9600 non-null   float64
 2   Blood Oxygen Level (%)  9700 non-null   float64
 3   Step Count              9900 non-null   float64
 4   Sleep Duration (hours)  9850 non-null   object 
 5   Activity Level          9800 non-null   object 
 6   Stress Level            9800 non-null   object 
dtypes: float64(4), object(3)
memory usage: 547.0+ KB
None
User ID                   201
Heart Rate (BPM)          400
Blood Oxygen Level (%)    300
Step Count                100
Sleep Duration (hours)    150
Activity Level            200
Stress Level              200
dtype: int64
    User ID  Heart Rate (BPM)  Blood Oxygen Level (%)    Step Count  \
0    4174.0         58.939776               98.809650   5450.39

##Data Cleansing-Standarisasi kolom user id


In [1064]:
data['User ID'].dtype

dtype('float64')

In [1065]:
#Mengubah tipe data User ID menjadi integer
data['User ID'] = data['User ID'].astype('Int64')

In [1066]:
data['User ID'].dtype

Int64Dtype()

##Data Cleansing-Standarisasi kolom active

In [1067]:
print(data['Activity Level'].unique())

['Highly Active' 'Highly_Active' 'Actve' 'Seddentary' 'Sedentary' 'Active'
 nan]


In [1068]:
# Buat kamus mapping untuk mengganti nilai yang salah
activity_mapping = {
    'Highly_Active': 'Highly Active',
    'Highly Active': 'Highly Active',
    'Actve': 'Active',
    'Active': 'Active',
    'Sedentary': 'Sedentary',
    'Seddentary': 'Sedentary'
}

# Terapkan mapping ke kolom Activity Level
data['Activity Level'] = data['Activity Level'].replace(activity_mapping)

In [1069]:
print(data['Activity Level'].unique())

['Highly Active' 'Active' 'Sedentary' nan]


##Data Cleansing-Standarisasi kolom stress level


In [1070]:
print(data['Stress Level'].unique())

['1' '5' '3' '6' '10' '2' '8' '9' nan '4' 'Very High' '7']


In [1071]:
#Mengganti nilai 'Very High' ke '10'
data['Stress Level'] = data['Stress Level'].replace({
    'Very High': '10'
})

In [1072]:
print(data['Stress Level'].unique())

['1' '5' '3' '6' '10' '2' '8' '9' nan '4' '7']


##Data Cleansing-Menghapus data kosong kolom user id

In [1073]:
print(data['User ID'].isnull().sum())
#Menghapus data dengan User ID kosong
data = data.dropna(subset=['User ID'])
print(f"Jumlah data setelah dihapus: {len(data)}")

201
Jumlah data setelah dihapus: 9799


##Data Cleansing-Deduplikasi data

In [1074]:
# Cek apakah ada data yang terduplikasi
data.duplicated().sum()

np.int64(0)

##Data Enrichment


###Heart Rate

In [1075]:
data['Heart Rate (BPM)'].isna().sum(), data['Heart Rate (BPM)'].describe()

(np.int64(394),
 count    9405.000000
 mean       76.026305
 std        19.386211
 min        40.000000
 25%        64.914467
 50%        75.208240
 75%        85.239576
 max       296.593970
 Name: Heart Rate (BPM), dtype: float64)

In [1076]:
# Ganti nilai tidak realistis menjadi NaN
data.loc[(data['Heart Rate (BPM)'] < 40) | (data['Heart Rate (BPM)'] > 200), 'Heart Rate (BPM)'] = np.nan

In [1077]:
# Mengisi nilai NaN dengan median
median_heart_rate = data['Heart Rate (BPM)'].median()
data['Heart Rate (BPM)'] = data['Heart Rate (BPM)'].fillna(median_heart_rate)

In [1078]:
print(data['Heart Rate (BPM)'].isna().sum())
print(data['Heart Rate (BPM)'].describe())


0
count    9799.000000
mean       75.134280
std        14.424232
min        40.000000
25%        65.354005
50%        75.134162
75%        84.580499
max       130.312002
Name: Heart Rate (BPM), dtype: float64


###Blood Oxygen Level

In [1079]:
data['Blood Oxygen Level (%)'].isna().sum(), data['Blood Oxygen Level (%)'].describe()

(np.int64(296),
 count    9503.000000
 mean       97.844952
 std         1.730815
 min        90.791208
 25%        96.666721
 50%        98.014441
 75%        99.374389
 max       100.000000
 Name: Blood Oxygen Level (%), dtype: float64)

In [1080]:
# Mengisi nilai NaN dengan median
median_spo2 = data['Blood Oxygen Level (%)'].median()
data['Blood Oxygen Level (%)'] = data['Blood Oxygen Level (%)'].fillna(median_spo2)

In [1081]:
data['Blood Oxygen Level (%)'].isna().sum(), data['Blood Oxygen Level (%)'].describe()

(np.int64(0),
 count    9799.000000
 mean       97.850072
 std         1.704717
 min        90.791208
 25%        96.709280
 50%        98.014441
 75%        99.326499
 max       100.000000
 Name: Blood Oxygen Level (%), dtype: float64)

###Step Count

In [1082]:
data['Step Count'].isna().sum(), data['Step Count'].describe()


(np.int64(98),
 count     9701.000000
 mean      7002.117395
 std       6908.154798
 min          0.910138
 25%       2021.265126
 50%       4977.574718
 75%       9750.114078
 max      62486.690753
 Name: Step Count, dtype: float64)

In [1083]:
# Mengganti nilai yang tidak wajar dengan NaN (Step Count < 100, Step COunt > 40000))
data.loc[(data['Step Count'] < 100) | (data['Step Count'] > 40000), 'Step Count'] = np.nan

In [1084]:
# Mengisi nilai NaN dengan median
median_steps = data['Step Count'].median()
data['Step Count'] = data['Step Count'].fillna(median_steps)


In [1085]:
# Mengganti tipe data menjadi integer
data['Step Count'] = data['Step Count'].astype('int64')

In [1086]:
data['Step Count'].isna().sum(),data['Step Count'].describe()

(np.int64(0),
 count     9799.000000
 mean      6925.752934
 std       6445.071796
 min        101.000000
 25%       2161.000000
 50%       5046.000000
 75%       9633.000000
 max      39882.000000
 Name: Step Count, dtype: float64)

###Sleep Duration (Hours)

In [1087]:
data['Sleep Duration (hours)'].isna().sum(), data['Sleep Duration (hours)'].describe()

(np.int64(145),
 count      9654
 unique     9415
 top       ERROR
 freq        240
 Name: Sleep Duration (hours), dtype: object)

In [1088]:
# Mengubah tipe data ke numeric dan ganti nilai yang bukan angka dengan NaN
data['Sleep Duration (hours)'] = pd.to_numeric(
    data['Sleep Duration (hours)'],
    errors='coerce'
)

In [1089]:
data['Sleep Duration (hours)'].isna().sum(), data['Sleep Duration (hours)'].describe()

(np.int64(385),
 count    9414.000000
 mean        6.503175
 std         1.510588
 min        -0.194453
 25%         5.488165
 50%         6.501197
 75%         7.529543
 max        12.140233
 Name: Sleep Duration (hours), dtype: float64)

In [1090]:
# Ubah nilai di luar rentang realistis jadi NaN
data.loc[(data['Sleep Duration (hours)'] < 2) |
         (data['Sleep Duration (hours)'] > 15), 'Sleep Duration (hours)'] = np.nan

In [1091]:
# Isi NaN dengan median
median_sleep = data['Sleep Duration (hours)'].median()
data['Sleep Duration (hours)'] = data['Sleep Duration (hours)'].fillna(median_sleep)

In [1092]:
data['Sleep Duration (hours)'].isna().sum(), data['Sleep Duration (hours)'].describe()

(np.int64(0),
 count    9799.000000
 mean        6.509886
 std         1.468938
 min         2.112415
 25%         5.542171
 50%         6.503308
 75%         7.476247
 max        12.140233
 Name: Sleep Duration (hours), dtype: float64)

###Activity Level

In [1093]:
data['Activity Level'].isna().sum(), data['Activity Level'].describe()

(np.int64(197),
 count          9602
 unique            3
 top       Sedentary
 freq           3251
 Name: Activity Level, dtype: object)

In [1094]:
# Isi dengan nilai paling sering muncul (mode)
mode_activity = data['Activity Level'].mode()[0]
data['Activity Level'] = data['Activity Level'].fillna(mode_activity)

In [1095]:
data['Activity Level'].isna().sum(), data['Activity Level'].describe()

(np.int64(0),
 count          9799
 unique            3
 top       Sedentary
 freq           3448
 Name: Activity Level, dtype: object)

###Stress Level

In [1096]:
data['Stress Level'].isna().sum(), data['Stress Level'].describe()

(np.int64(196),
 count     9603
 unique      10
 top          2
 freq       992
 Name: Stress Level, dtype: object)

In [1097]:
data['Stress Level'].unique()

array(['1', '5', '3', '6', '10', '2', '8', '9', nan, '4', '7'],
      dtype=object)

In [1098]:
# Mengubah tipe data ke numeric dan ganti nilai yang bukan angka dengan NaN
data['Stress Level'] = pd.to_numeric(
    data['Stress Level'],
    errors='coerce'
)

In [1099]:
print(data['Stress Level'].dtype)

float64


In [1100]:
# Isi NaN dengan median
median_stressLevel = data['Stress Level'].median()
data['Stress Level'] = data['Stress Level'].fillna(median_stressLevel)
data['Stress Level'] = data['Stress Level'].astype(int)

In [1101]:
print(data['Stress Level'].dtype)

int64


In [1102]:
data['Stress Level'].isna().sum(), data['Stress Level'].describe()

(np.int64(0),
 count    9799.000000
 mean        5.475967
 std         2.852717
 min         1.000000
 25%         3.000000
 50%         5.000000
 75%         8.000000
 max        10.000000
 Name: Stress Level, dtype: float64)

##Hasil

In [1103]:
print(data)

      User ID  Heart Rate (BPM)  Blood Oxygen Level (%)  Step Count  \
0        4174         58.939776               98.809650        5450   
2        1860         75.134162               97.052954        2826   
3        2294         40.000000               96.894213       13797   
4        2130         61.950165               98.583797       15679   
5        2095         96.285938               94.202910       10205   
...       ...               ...                     ...         ...   
9995     1524         78.819386               98.931927        2948   
9996     4879         48.632659               95.773035        4725   
9997     2624         73.834442               97.945874        2571   
9998     4907         75.134162               98.401058        3364   
9999     4113         70.063864               98.475606         544   

      Sleep Duration (hours) Activity Level  Stress Level  
0                   7.167236  Highly Active             1  
2                   6.50330

In [1104]:
print(data.info())
print(data.isnull().sum())
print(data.head(25))

<class 'pandas.core.frame.DataFrame'>
Index: 9799 entries, 0 to 9999
Data columns (total 7 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   User ID                 9799 non-null   Int64  
 1   Heart Rate (BPM)        9799 non-null   float64
 2   Blood Oxygen Level (%)  9799 non-null   float64
 3   Step Count              9799 non-null   int64  
 4   Sleep Duration (hours)  9799 non-null   float64
 5   Activity Level          9799 non-null   object 
 6   Stress Level            9799 non-null   int64  
dtypes: Int64(1), float64(3), int64(2), object(1)
memory usage: 622.0+ KB
None
User ID                   0
Heart Rate (BPM)          0
Blood Oxygen Level (%)    0
Step Count                0
Sleep Duration (hours)    0
Activity Level            0
Stress Level              0
dtype: int64
    User ID  Heart Rate (BPM)  Blood Oxygen Level (%)  Step Count  \
0      4174         58.939776               98.809650        5450